# Step 1: Import Libraries & Set Processing Device


In [1]:

# Libraries

import os
from time import time
from tqdm import tqdm
import numpy as np

import torch
from torch.nn import Linear, CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet50_Weights
from torchvision.models import resnet50
from torchvision.transforms import transforms
import torch
from torch.utils.data import random_split
from torch.cuda.amp import GradScaler, autocast


# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Step 2: Load Data
#### A. Data Transformer
#### B. Create Dataset
#### C. Data Loader

In [2]:
# 训练数据的 transforms
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.RandomAffine(degrees=0,translate=(0.05,0.05)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.RandomRotation(degrees=15),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 测试数据的 transforms
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])



In [3]:
train_set = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=32,
                                          shuffle=True, num_workers=2 ,
                                          pin_memory=True)

test_set = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform_test)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=32,
                                         shuffle=False, num_workers=2 , 
                                         pin_memory=True)



Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_size = len(train_set)
val_size = len(test_set)
print(len(train_set))
print(len(test_set))


50000
10000


# Step 3: Build Model, Optimiser and Loss Function

In [5]:
# Model
PATH = "../model/ResNet50_cifar100.pth"
model = resnet50()
# print(model)

last_fc = model.fc.in_features
model.fc = Linear(last_fc, 100)

model.load_state_dict(torch.load(PATH))

model = model.to(device)
# Set the model to evaluation mode
model.eval()
print(model)

# model = model.cuda()    # only if your system supports Nvidia CUDA

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
# Optimiser
optimiser = Adam(model.parameters(), lr=3e-4, weight_decay=0.0001)

# Loss Function
loss_fn = CrossEntropyLoss()

# Step 4: Train & Evaluate the Model


In [7]:
EPOCH = 15

In [8]:
scaler = GradScaler()

for epoch in range(EPOCH):
    print(f"\nEpoch: {epoch}")
    
    start = time()
    
    tr_acc = 0
    test_acc = 0
    
    # Train
    model.train()
    
    with tqdm(train_loader, unit="batch" , desc='Training') as tepoch:
        for xtrain, ytrain in tepoch:
            torch.cuda.empty_cache()
            optimiser.zero_grad()            
            
            xtrain , ytrain_gpu = xtrain.to(device) ,  ytrain.to(device)
            
            # Step 6: Perform forward pass and calculate loss
            with autocast():
                train_prob = model(xtrain)
                loss = loss_fn(train_prob, ytrain_gpu)

           
            scaler.scale(loss).backward()
            scaler.step(optimiser)
            scaler.update()
            
            # training ends
            train_prob = train_prob.cpu()
            
            train_pred = torch.max(train_prob, 1).indices
            tr_acc += int(torch.sum(train_pred == ytrain))
            
        ep_tr_acc = tr_acc / train_size
    
    # Evaluate
    model.eval()
    with torch.no_grad():
        with tqdm(test_loader, unit="batch" , desc='Testing') as test_epoch:
            for xtest, ytest in test_epoch:
                torch.cuda.empty_cache()
                xtest = xtest.to(device)
                test_prob = model(xtest)
                test_prob = test_prob.cpu()
                
                test_pred = torch.max(test_prob,1).indices
                test_acc += int(torch.sum(test_pred == ytest))
                
            ep_test_acc = test_acc / val_size
    
    end = time()
    duration = (end - start) / 60
    
    print(f"Time: {duration}, Loss: {loss:.2f}\nTrain_acc: {ep_tr_acc*100 :.2f}, Test_acc: {ep_test_acc*100 :.2f}")


Epoch: 0


Testing: 100%|██████████| 313/313 [00:32<00:00,  9.53batch/s]


Time: 4.897655725479126, Loss: 1.29
Train_acc: 64.34, Test_acc: 58.75

Epoch: 1


Testing: 100%|██████████| 313/313 [00:33<00:00,  9.45batch/s]


Time: 4.845637627442678, Loss: 1.65
Train_acc: 65.15, Test_acc: 58.54

Epoch: 2


...

Time: 4.801262684663137, Loss: 1.12
Train_acc: 65.46, Test_acc: 59.23

Epoch: 3


Testing: 100%|██████████| 313/313 [00:32<00:00,  9.71batch/s]


Time: 4.777305181821188, Loss: 1.00
Train_acc: 66.29, Test_acc: 59.08

Epoch: 4


Testing: 100%|██████████| 313/313 [00:32<00:00,  9.70batch/s]

Time: 4.764993989467621, Loss: 1.23
Train_acc: 66.48, Test_acc: 59.61


In [9]:

torch.save(model.state_dict(), PATH)
